In [ ]:
import os
import time

import cv2

from nanodet.data.transform import Pipeline
from nanodet.util import cfg, load_config
from nanodet.util.path import mkdir

image_ext = [".jpg", ".jpeg", ".webp", ".bmp", ".png"]
video_ext = ["mp4", "mov", "avi", "mkv"]

def get_image_list(path):
    image_names = []
    for maindir, subdir, file_name_list in os.walk(path):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            ext = os.path.splitext(apath)[1]
            if ext in image_ext:
                image_names.append(apath)
    return image_names

config = "/home/tao/Github/nanodet_custom/config/gray_stereo.yml"
path = "/home/tao/Pictures/real_data_stereo_ann/data"
save_result = True

local_rank = 0

load_config(cfg, config)
pipeline = Pipeline(cfg.data.train.pipeline, cfg.data.train.keep_ratio)
current_time = time.localtime()

if os.path.isdir(path):
    files = get_image_list(path)
else:
    files = [path]
files.sort()
for image_name in files:
    img_info = {"id": 0}
    if isinstance(image_name, str):
        img_info["file_name"] = os.path.basename(image_name)
        img = cv2.imread(image_name)
    else:
        img_info["file_name"] = None

    height, width = img.shape[:2]
    img_info["height"] = height
    img_info["width"] = width
    meta = dict(img_info=img_info, raw_img=img, np_img=img, img=img)
    meta = pipeline(None, meta, cfg.data.train.input_size)
    if save_result:
        save_folder = os.path.join(
            cfg.save_dir, time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
        )
        mkdir(local_rank, save_folder)
        save_file_name = os.path.join(save_folder, os.path.basename(image_name))
        result_image = meta["img"]
        cv2.imwrite(save_file_name, result_image)
        print(result_image.shape)
